# Mundo 13 - Interpolação curva de juros

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
from datetime import datetime
from pandas.tseries.offsets import *

In [ ]:
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install())) 

In [ ]:
data_di = "06/03/2019"
mercadoria = 'DI1'

In [ ]:
url = f'''
https://www2.bmf.com.br/pages/portal/bmfbovespa/boletim1/SistemaPregao1.asp?pagetype=pop&caminho=Resumo%20
Estat%EDstico%20-%20Sistema%20Preg%E3o&Data={data_di}&Mercadoria={mercadoria}
'''

driver.get(url)

driver.implicitly_wait(3)

'''
esse método de congelar é MUITO mais eficiente que o time, porque ele só espera o tempo necessário.
se o elemento que você quer achar já carregou, ele roda e acabou. 
'''

driver.maximize_window()

#driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options = options) 


local_tabela = '''
/html/body/div/div[2]/form[1]/table[3]/tbody/tr[3]/td[3]/table
'''

local_indice = '''
/html/body/div/div[2]/form[1]/table[3]/tbody/tr[3]/td[1]/table
'''

elemento = driver.find_element("xpath", local_tabela)

#pode ser ID, class_, XPATH ou name (vc so vai usar um dos 4)

elemento_indice = driver.find_element("xpath", local_indice)

html_tabela = elemento.get_attribute('outerHTML')
html_indice = elemento_indice.get_attribute('outerHTML')

tabela = pd.read_html(html_tabela)[0]
indice = pd.read_html(html_indice)[0]

driver.quit()

tabela.columns = tabela.loc[0]

tabela = tabela['ÚLT. PREÇO']

tabela = tabela.drop(0, axis = 0)

indice.columns = indice.loc[0]

indice_di = indice['VENCTO']

indice = indice.drop(0, axis = 0)

tabela.index = indice['VENCTO']

tabela = tabela.astype(int)

tabela = tabela[tabela != 0]

tabela = tabela/1000

legenda = pd.Series(['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'],
                    index = ['F', 'G', 'H', 'J', 'K', 'M', 'N', 'Q', 'U', 'V', 'X', 'Z'])

lista_datas = []

for indice in tabela.index:

    letra = indice[0]
    
    legenda["F"]
    ano = indice[1:3]

    mes = legenda[letra]

    data = f"{mes}-{ano}"

    data = datetime.strptime(data, "%b-%y")

    lista_datas.append(data)


tabela.index = lista_datas  

tabela

print(tabela)

# Interpolação

Vamos esse cenário:

Hoje é dia 06/03/2019.
Essa é a curva de juros.
Qual é o DI de 18/04 de 2023?

In [ ]:
curva_dias_uteis = []
dia_atual = datetime(2019, 3, 6)

for dia in tabela.index:
    
    dias_uteis = len(pd.date_range(dia_atual, dia, freq=BDay()))
    
    curva_dias_uteis.append(dias_uteis)
    
curva_dias_uteis

In [ ]:
taxas = tabela.values

taxas = list(taxas)

taxas

In [ ]:
len(curva_dias_uteis)

In [ ]:
dias_uteis_daqui_pra_data = len(pd.date_range(dia_atual, datetime(2023, 4, 18), freq=BDay()))

dias_uteis_daqui_pra_data

In [ ]:
from scipy import interpolate
import mplcyberpunk
import matplotlib.pyplot as plt

plt.style.use("cyberpunk")

In [ ]:
linear = interpolate.interp1d(curva_dias_uteis, taxas, kind = 'linear')
cubica = interpolate.interp1d(curva_dias_uteis, taxas, kind = 'cubic')

In [ ]:
dias_novos = [dias_uteis_daqui_pra_data, 520, 385, 2200]
taxas_linear = list(linear(dias_novos))
taxas_cubica = list(cubica(dias_novos))

In [ ]:
taxas_linear

In [ ]:
fig, ax = plt.subplots()

ax.scatter(curva_dias_uteis, taxas)
ax.scatter(dias_novos, taxas_linear)
ax.scatter(dias_novos, taxas_cubica)

